# Building a Conversational Customer Support Agent with AutoGen

In [ ]:
!pip install pyautogen

In [4]:
import os
from getpass import getpass

from autogen import ConversableAgent, GroupChatManager

In [5]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [50]:
human_proxy = ConversableAgent(
    name="human_proxy",
    description="A customer who is seeking help from a customer support agent.",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

frontman_agent = ConversableAgent(
    name="frontman_agent",
    description="A customer support agent who interacts with the human_proxy in a conversational manner to gather information from and relay messages back to them.",
    system_message="You are a customer support agent who interacts with customers in a conversational manner to help them solve their problems.\
Your job is to gather information from the customer until you've identified a clear intent behind their problem, at which time, you return the customers intent.\
If you are unable to identify the intent, you should ask the customer for more information.\
You are also responsible for relaying messages back to the customer.",
    llm_config={
        "config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
    },
    human_input_mode="NEVER",
)

intent_router_agent = ConversableAgent(
    name="intent_router_agent",
    description="A tool to classify a user's intent into pre-defined categories.",
    system_message="""Your job is to classify a users intent into one of the following categories:

[1] Check account balance
[2] Cancel credit card
[3] Open a new account

You should return the intent number that best matches the user's intent. If there is no clear match, you should return [NONE].""",
    llm_config={
        "config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
    },
    human_input_mode="NEVER",
)

In [53]:
from autogen import GroupChat

group_chat = GroupChat(
    agents=[frontman_agent, intent_router_agent, human_proxy],
    allowed_or_disallowed_speaker_transitions={
        human_proxy: [frontman_agent],
        frontman_agent: [human_proxy, intent_router_agent],
    },
    speaker_transitions_type="allowed",
    messages=[],
    speaker_selection_method="auto",
    max_round=6,
    send_introductions=True,
)

group_chat_manager = GroupChatManager(
    name="group_chat_manager",
    groupchat=group_chat,
    llm_config={
        "config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]
    },
)

In [54]:
result = human_proxy.initiate_chat(
    group_chat_manager,
    # message="Hello, I need help checking my checking account balance.",
    message="Hello. I'm andrew",
)

human_proxy (to group_chat_manager):

Hello. I'm andrew

--------------------------------------------------------------------------------

Next speaker: frontman_agent

frontman_agent (to group_chat_manager):

Hello Andrew! How can I assist you today?

--------------------------------------------------------------------------------

Next speaker: human_proxy

human_proxy (to group_chat_manager):

I want to know how much money is in my checking account

--------------------------------------------------------------------------------

Next speaker: frontman_agent

frontman_agent (to group_chat_manager):

Got it, Andrew. To help you with that, I will need some additional information. Could you please provide your account number and confirm your identity (such as your name and any security questions or codes that may be required)?

--------------------------------------------------------------------------------

Next speaker: human_proxy



In [10]:
result.chat_history

[{'content': 'Hello, I need help checking my checking account balance.',
  'role': 'assistant'},
 {'content': "Hello! I'd be happy to help you with that. Could you please provide me with your full name and account number? Additionally, may I ask for a way to verify your identity, such as the last four digits of your Social Security number or a recent transaction amount?",
  'role': 'user'},
 {'content': 'my name is andrew', 'role': 'assistant'},
 {'content': 'Thank you, Andrew. For security purposes, I need a bit more information to verify your identity. Could you please provide either:\n\n1. The last four digits of your Social Security number, or\n2. A recent transaction amount and the date it occurred.\n\nOnce I have that information, I can assist you with checking your account balance.',
  'role': 'user'}]

In [14]:
result.cost

{'usage_including_cached_inference': {'total_cost': 0.006634999999999999,
  'gpt-4o-2024-05-13': {'cost': 0.006634999999999999,
   'prompt_tokens': 505,
   'completion_tokens': 274,
   'total_tokens': 779}},
 'usage_excluding_cached_inference': {'total_cost': 0.00558,
  'gpt-4o-2024-05-13': {'cost': 0.00558,
   'prompt_tokens': 456,
   'completion_tokens': 220,
   'total_tokens': 676}}}